# GitHub data collection

---

## Preparation & test
Loading packages & stuff.

In [0]:
#!pip install PyGithub

In [0]:
import csv
import json
import os
import pickle
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from github import Github
from multiprocessing import Pool, cpu_count
from random import randint, uniform
from time import sleep
from tqdm import tqdm_notebook as tqdm
from urllib.request import urlopen, Request
from urllib.parse import urlparse

In [0]:
g = Github(client_id='INSERT',  # insert
           client_secret='INSERT')

In [0]:
PATH_TO_DATA = 'INSERT'

In [0]:
# чтобы варнинги из будущего не беспокоили
# cuz im a dangerous woman
import warnings
warnings.filterwarnings('ignore')

# Structure

---

## List of repositories to collect
### Top open source projects
**Description from The State of the Octoverse**: *Top 10 open source projects by number of unique contributors between October 1, 2017 and September 30, 2018.*

1.	[Microsoft/vscode](https://github.com/Microsoft/vscode)	19K
2.	[facebook/react-native](https://github.com/facebook/react-native)	10K
3.	[tensorflow/tensorflow](https://github.com/tensorflow/tensorflow)	9.3K
4.	[angular/angular-cli](https://github.com/angular/angular-cli)	8.8K
5.	[MicrosoftDocs/azure-docs	](https://github.com/MicrosoftDocs/azure-docs)7.8K
6.	[angular/angular](https://github.com/angular/angular)	7.6K
7.	[ansible/ansible](https://github.com/ansible/ansible)	7.5K
8.	[kubernetes/kubernetes](https://github.com/kubernetes/kubernetes)	6.5K
9.	[npm/npm](https://github.com/npm/npm)	6.1K
10.	[DefinitelyTyped/DefinitelyTyped](https://github.com/DefinitelyTyped/DefinitelyTyped)	6.0K


### Fastest growing open source projects
**Description from The State of the Octoverse**: 
*Top 10 open source projects by percent increase in contributors. This list is limited to projects that had more than 1,000 contributors in 2017—and doesn’t include projects used in MOOCs and bootcamps.*
1.	[MicrosoftDocs/azure-docs](https://github.com/MicrosoftDocs/azure-docs)	4.7X
2.	[pytorch/pytorch](https://github.com/pytorch/pytorch)	2.8X
3.	[godotengine/godot](https://github.com/godotengine/godot)	2.2X
4.	[nuxt/nuxt.js](https://github.com/nuxt/nuxt.js)	2.1X
5.	[ethereum/go-ethereum](https://github.com/ethereum/go-ethereum)	2.0X
6.	[wix/react-native-navigation](https://github.com/wix/react-native-navigation)	1.9X
7.	[spyder-ide/spyder](https://github.com/spyder-ide/spyder)	1.8X
8.	[tensorflow/models](https://github.com/tensorflow/models)	1.8X
9.	[home-assistant/home-assistant	](https://github.com/home-assistant/home-assistant)1.6X
10.	[MarlinFirmware/Marlin](https://github.com/MarlinFirmware/Marlin)	1.6X


In [0]:
# create URL list
gh_urls = ['Microsoft/vscode', 'facebook/react-native', 'tensorflow/tensorflow', 
           'angular/angular-cli', 'MicrosoftDocs/azure-docs', 'angular/angular',
           'ansible/ansible', 'kubernetes/kubernetes', 'npm/npm', 'DefinitelyTyped/DefinitelyTyped',
           'MicrosoftDocs/azure-docs', 'pytorch/pytorch', 'godotengine/godot', 
           'nuxt/nuxt.js', 'ethereum/go-ethereum', 'wix/react-native-navigation',
           'spyder-ide/spyder', 'tensorflow/models', 'home-assistant/home-assistant',
           'MarlinFirmware/Marlin']

In [0]:
tosp_gh_urls = ['Microsoft/vscode', 'facebook/react-native', 'tensorflow/tensorflow', 
           'angular/angular-cli', 'MicrosoftDocs/azure-docs', 'angular/angular',
           'ansible/ansible', 'kubernetes/kubernetes', 'npm/npm', 'DefinitelyTyped/DefinitelyTyped']
fgosp_gh_urls = ['MicrosoftDocs/azure-docs', 'pytorch/pytorch', 'godotengine/godot', 
           'nuxt/nuxt.js', 'ethereum/go-ethereum', 'wix/react-native-navigation',
           'spyder-ide/spyder', 'tensorflow/models', 'home-assistant/home-assistant',
           'MarlinFirmware/Marlin']

# Repositories
---
### Brief overview of methods & test

In [0]:
# return methods (without _)
def small_dir(obj):
  return [x for x in dir(obj) if not x.startswith('_')]

In [0]:
test_urls = ['PyGithub/PyGithub', 'actuallyykatie/github-research'] 

In [0]:
data = []
for url in test_urls:
  # get repo
  tmp_repo = g.get_repo(url)
  data.append(dict(zip(['name', 'description', 'owner', 'language'],
                       [tmp_repo.name, tmp_repo.description, tmp_repo.owner.login, tmp_repo.language])))


In [0]:
pd.DataFrame(columns=['name', 'description', 'owner', 'language']).append(data)  # it works!

,name,description,owner,language
0,PyGithub,Typed interactions with the GitHub API v3,PyGithub,Python
1,github-research,Source code and files for research paper on An...,actuallyykatie,Jupyter Notebook


## Data Collection
Date: 30/04/2019  
- **full_name** - full name of the repository (owner/title)  
- **name** - title of repository  
- **description**  - description 
- **owner** - owner  
- **language** - programming languages used
- **forks_count **- number of forks
- **stargazers_count **- number of stargazers  
- **watchers_count **- number of watchers  
- **open_issues_count** - number of open issues  
- **topics** - topics (like tags)  
- **contributors** - list of contributors 

*Note:* MicrosoftDocs/azure-docs is listed twice as it appears in both top-10s.

In [0]:
repos_data = []
for url in gh_urls:
  # get repo
  tmp_repo = g.get_repo(url)
  # append data 
  repos_data.append(dict(zip(['full_name','name', 'description', 'owner', 'language', 
                              'forks_count','stargazers_count', 'watchers_count', 
                              'open_issues_count', 'topics', 'contributors'],
                             [tmp_repo.full_name, tmp_repo.name, tmp_repo.description, tmp_repo.owner.login, tmp_repo.language,
                              tmp_repo.forks_count, tmp_repo.stargazers_count, tmp_repo.watchers, 
                              tmp_repo.open_issues_count,  tmp_repo.get_topics(), 
                              [name.login for name in tmp_repo.get_contributors()] 
                             ])))

In [19]:
repo_cols = ['full_name','name', 'description', 'owner', 'language', 
             'forks_count','stargazers_count', 'watchers_count', 
             'open_issues_count', 'topics', 'contributors']
repos_df = pd.DataFrame(repos_data, columns=repo_cols)
repos_df  

,full_name,name,description,owner,language,forks_count,stargazers_count,watchers_count,open_issues_count,topics,contributors
0,Microsoft/vscode,vscode,Visual Studio Code,Microsoft,TypeScript,10174,74535,74535,5727,"[editor, electron, visual-studio-code, typescr...","[bpasero, joaomoreno, jrieken, isidorn, alexan..."
1,facebook/react-native,react-native,A framework for building native apps with React.,facebook,JavaScript,17081,76546,76546,483,[],"[shergin, javache, davidaurelio, mkonicek, sah..."
2,tensorflow/tensorflow,tensorflow,An Open Source Machine Learning Framework for ...,tensorflow,C++,74224,126592,126592,2471,"[tensorflow, machine-learning, python, deep-le...","[tensorflower-gardener, caisq, benoitsteiner, ..."
3,angular/angular-cli,angular-cli,CLI tool for Angular,angular,TypeScript,6494,21509,21509,717,"[angular, cli, angular-cli, typescript]","[hansl, filipesilva, clydin, Brocco, alan-agiu..."
4,MicrosoftDocs/azure-docs,azure-docs,Open source documentation of Microsoft Azure,MicrosoftDocs,PowerShell,7000,2524,2524,1912,[],"[tysonn, v-alje, GitHubber17, ktoliver, Ja-Dun..."
5,angular/angular,angular,One framework. Mobile & desktop.,angular,TypeScript,12694,47609,47609,2751,"[angular, typescript, web, javascript, pwa, we...","[petebacondarwin, vsavkin, vicb, IgorMinar, gk..."
6,ansible/ansible,ansible,Ansible is a radically simple IT automation pl...,ansible,Python,15259,36843,36843,5643,"[python, ansible]","[mpdehaan, bcoca, jimi-c, abadger, mattclay, d..."
7,kubernetes/kubernetes,kubernetes,Production-Grade Container Scheduling and Mana...,kubernetes,Go,17933,51962,51962,3189,"[kubernetes, go, cncf, containers]","[k8s-github-robot, k8s-ci-robot, brendandburns..."
8,npm/npm,npm,This repository is moving to: https://github.c...,npm,None,3161,17134,17134,2166,[],"[isaacs, iarna, othiym23, zkat, robertkowalski..."
9,DefinitelyTyped/DefinitelyTyped,DefinitelyTyped,The repository for high quality TypeScript typ...,DefinitelyTyped,TypeScript,17323,21842,21842,2809,"[typescript, definition, dts, types, typings, ...","[vvakame, sandersn, DanielRosenwasser, andy-ms..."


In [0]:
repos_df.to_csv(PATH_TO_DATA + 'repos_df.csv')

---

# Issues

---

Date: 25/04/2019 to 30/04/2019

- repo_author - Repository author (company name)
- repo_title - Repository title
- issue_title - Issue/PR title
- issue_number - Issue number
- state - State
- rev_or_ass - Reviewers/Assignees
- labels - Labels
- created_at - Created at
- closed_at - Who closed the issue/PR and when (if so)
- num_of_participants - Number of participants
- comments - Comments. Includes the user's status (collaborator/author/owner/member), user login, date, comment
- reactions - Reactions

**GitHub rate limit:** 5000 requests per hour.   
- but we have a LOT of issues  
- limit is reached quite easily

In order to overcome it, we need to:  
- scrap it, luckily github allows that

Processing one issue by one was too slow, so we also added multiprocessing. The functions used to return the pandas dataframes, but due to Pool + map we had to convert it into lists, and then back to pandas. :(.

# Functions
The plan is like that:
1. For each repository create a dataframe, where each comment is a row
2. Return this row as a list (cleaned from stuff that may further break csv, like commas)   
3. Combine all the lists
4. Then turn back to pandas

Result: one dataset for each repository. Each row - comment.   
*Note*: issue description also counts as a comment, so we do not skip issues that have zero comments by other users *(but we kinda don't need them anyway?)*.

In [0]:
def get_reactions(soup):
  """
  Gets reactions for each comment if any.
  (Что может быть более костыльным?)
  """
  i=0
  comm_reacts = []
  for x in soup.find_all(class_='comment-reactions'):
    if len(x.contents) < 2:
      comm_reacts.append(x.contents)
    else:
      comm_reacts.append([y.text for y in soup.find_all('form', {'class':'js-pick-reaction'})][i])
      i+=1
  return comm_reacts

In [0]:
def get_comments_df(soup):
  """
  Gets comments for the issue.
  Number of comments for issues and number of reactions sometimes does not match
  in case of PRs, so if error occurs, function fills the column with '\n'.
  
  На пулл реквестах число комментов и реакций не совпадает, 
  т.к. не всё что с реакцией - коммент (при этом тэг один).
  В итоге если всё окей, то в дф пишутся реакции. Если нет, то просто '\n'
  """
  comms = clean_comments(soup)
  try:
    comments_df = pd.DataFrame.from_dict({
        'issue_title': soup.find(class_='js-issue-title').text.strip().replace(',', ''), # 
        'comment_number': [n for n in np.arange(1, len(comms)+1, 1)],
        'comments': comms,
        'reactions': get_reactions(soup)})
  except ValueError:
    comments_df = pd.DataFrame.from_dict({
        'issue_title': soup.find(class_='js-issue-title').text.strip().replace(',', ''),  #
        'comment_number': [n for n in np.arange(1, len(comms)+1, 1)],
        'comments': comms,
        'reactions': '\n'})
  return comments_df           

In [0]:
def clean_rev_ass(soup):
  """
  Clean data about Assignees & Reviewers. 
  All of them are located under the same tag. Issues have Assignees, PRs - Reviewers.
  This function deletes '\n', irrelevant text and some more text :))
  Returns logins splitted by ;
  """
  to_clean = soup.find(class_='js-issue-sidebar-form').text.replace('\n', '').strip()
  # rev_ass_cleaned = to_clean.replace('Assignees   ', '') if 'Assignees' in to_clean else to_clean.replace('Reviewers   ', '')
  
  if 'Assignees' in to_clean:
    rev_ass_cleaned = to_clean.replace('Assignees   ', '')
  else:
    rev_ass_cleaned = to_clean.replace('Reviewers   ', '')
    if 'At least 1 approving review is required to merge this pull request.' in rev_ass_cleaned :
      rev_ass_cleaned = rev_ass_cleaned.replace('At least 1 approving review is required to merge this pull request.', '')

      
  return '; '.join(rev_ass_cleaned.split())

In [0]:
def clean_closed_at(soup):
  """
  Clean and extract data about who and when closed the issue.
  """
  tmp_lst = flatten([ x.text.split('\n    ') for x in soup.find_all(class_='discussion-item discussion-item-closed')])
  #who_did, what_did, when_did = [x.replace('\n', '').strip() for x in tmp_lst ]
  closed_at = [x.replace('\n', '').strip() for x in tmp_lst ]

  return '; '.join(closed_at).replace(',', '')

In [0]:
def clean_comments(soup):
  """
  Remove commas, lots of '\n', irrelevant text, and return more or less clear comments.
  However, more info extraction is needed after that.
  """
  comm_dirty = [x.text.strip() for x in soup.find_all(class_='timeline-comment-group js-minimizable-comment-group js-targetable-comment')]
  comm_cleaned = [x.replace('\n', ' ').replace(',', '').replace('This comment has been minimized.','')\
                  .replace('Sign in to view', '').replace('Copy link', '')\
                  .replace('Quote reply', '').replace('   ',' ') for x in comm_dirty]
  # add extra word for the first element which is issue body
  comm_cleaned = [x.strip() for x in comm_cleaned]
  
  return comm_cleaned

In [0]:
cols = ['repo_author', 'repo_title', 'issue_title', 'issue_number', 'state', 
        'rev_or_ass', 'labels', 'created_at', 'closed_at',
        'num_of_participants']

In [0]:
def parse_issue(url):
  """
  Gets information about:
  1. Repository author
  2. Repository title
  3. Issue/PR title
  4. Issue number
  5. State
  6. Reviewers/Assignees
  7. Labels
  8. Created at
  9. Who closed the issue/PR, Closed at
  10. Number of participants
  11. Comments
  12. Reactions
  
  Used to return dataframe, now returns list. Version for pool & map ¯\_(ツ)_/¯
  """
  
  r = requests.get(url)
  sleep(uniform(0.1, 0.5))
  
  df_tmp = pd.DataFrame(columns=cols)
  
  if r.status_code == 200:
    try:
      #print('Processing ' + url)
      html = r.text
      soup = BeautifulSoup(html,  'lxml')  
      _trash, repo_author, repo_title, _issue, issue_number =  urlparse(url).path.strip().split('/') 
      # collect information about issue, turn into df
      issue_info = {
          'repo_author': repo_author,
          'repo_title': repo_title,
          'issue_title': soup.find(class_='js-issue-title').text.strip().replace(',', ''), # CHANGED
          'issue_number': issue_number,
          'state': soup.find(class_='TableObject-item').text.strip(),
          'rev_or_ass': clean_rev_ass(soup), ### changed
          'labels': '; '.join(soup.find(class_='discussion-sidebar-item sidebar-labels js-discussion-sidebar-item').text.strip('\n\n    Labels\n  \n\n').split('\n')),
          'created_at': datetime.strptime(soup.find(class_='timestamp js-timestamp').text, '%b %d, %Y').strftime('%d/%m/%Y'),
          'closed_at': clean_closed_at(soup),
          'num_of_participants': soup.find(class_='participation').text.strip()
      }
      issue_info_df_tmp = pd.DataFrame([issue_info], 
                                       columns= ['repo_author', 'repo_title', 'issue_title', 'issue_number', 'state', 
                                                 'rev_or_ass', 'labels', 'created_at', 'closed_at', 'num_of_participants'])
      # collect comments 
      comm_info = get_comments_df(soup)
      
      # merge it all
      df_tmp = pd.merge(issue_info_df_tmp, comm_info, on='issue_title', sort=False)
    except:
      pass
  
  return [str(x)  for x in df_tmp.values.tolist()]


In [0]:
def get_issue_links(repo):
  """
  Generate links to parse
  repo = 'author/repo_name'
  """
  tmp_repo = g.get_repo(repo)
  tmp_issues = tmp_repo.get_issues(state='all')
  issues_urls = ['https://github.com/{}/issues/{}'.format(repo, i) for i in range(1, tmp_issues[0].number + 1)] # 0 element is the last issue
  
  return issues_urls

In [0]:
# turn list of lists into one
flatten = lambda l: [item for sublist in l for item in sublist]

## A bit of tests & thoughts
In this section some tests are performed in order to make sure that everything works as it should. Also - some descriptions of why something is done the way it is done.

#### title & closed at

- commas may ruin the structure --> need to delete them
- need to split the 'closed at' into who did it and when *(here separated by ; , later will be splitted)*

```
.replace('\\n', ' ').replace(',', '')
```



In [0]:
r_test = requests.get('https://github.com/tensorflow/models/issues/1')
soup_test = BeautifulSoup(r_test.text,  'lxml')

In [0]:
soup_test.find(class_='js-issue-title').text.strip().replace(',', '')  # works, comma is deleted

'initial commit simple separated models'

In [0]:
r_test_tf = requests.get('https://github.com/tensorflow/models/pull/408')
soup_test_tf = BeautifulSoup(r_test_tf.text,  'lxml')

In [0]:
r_tf = requests.get('https://github.com/tensorflow/models/pull/250')
soup_tf = BeautifulSoup(r_tf.text, 'lxml')

In [0]:
soup_test_tf.find(class_='js-issue-title').text.strip().replace(',', '')

'Python 3 support for download_and_convert_data.py and add CIFAR100'

#### Assignees & Reviewers
Issues have "Assignees", PRs - "Reviewers".  The code section also contains some irrelevant information (like *"At least 1 approving review is required to merge this pull request."*) which we also need to remove.  

In [0]:
soup_test.find(class_='js-issue-sidebar-form').text.replace('\n', '').strip().replace('Assignees   ', '')

'actuallyykatie'

In [0]:
soup_tf.find(class_='js-issue-sidebar-form').text.replace('\n', '').strip()

'Reviewers   calberti      At least 1 approving review is required to merge this pull request.'

In [0]:
soup_test_tf.find(class_='js-issue-sidebar-form').text.replace('\n', '').strip().split()

['Reviewers', 'tremblerz', 'MarkDaoust', 'sguada']

**Examples** of how `clead_rev_ass` works:

In [0]:
# tests
print('Multiple reviewers ---> '+ clean_rev_ass(soup_test_tf))
print('Issue with one assignee ---> ' + clean_rev_ass(soup_test))
print('Single reviewer ---> ' + clean_rev_ass(soup_tf))

Multiple reviewers ---> tremblerz; MarkDaoust; sguada
Issue with one assignee ---> actuallyykatie
Single reviewer ---> calberti


#### labels processing 
Чтобы лейблы не сломались, нужно сначала разбить их по \n, потом склеить обратно через ;. Запятая не очень, потому что может в дальнейшем поломать csv :c
In order not to break labels structure as some of them contain spaces, we first need to split it by \n, then merge by ; *(not comma because we use , as separator while converting it back to csv df)*

In [0]:
soup_tf.find(class_='discussion-sidebar-item sidebar-labels js-discussion-sidebar-item').text

'\n\n    Labels\n  \n\ncla: yes\nstat:awaiting owner\n\n'

In [0]:
soup_tf.find(class_='discussion-sidebar-item sidebar-labels js-discussion-sidebar-item').text.split('\n')

['Labels', '  ', '', 'cla: yes', 'stat:awaiting owner']

Пустые штуки между ; прям бесят. С ними тоже разберемся.  
Вывод: можно удалить первую часть строки, она же не нужна!

--> Need to remove the beginning with "'Labels;   ; ; "

In [0]:
'; '.join(soup_tf.find(class_='discussion-sidebar-item sidebar-labels js-discussion-sidebar-item').text.strip().split('\n'))
#'Labels;   ; ; 

'Labels;   ; ; cla: yes; stat:awaiting owner'

In [0]:
'; '.join(soup_test.find(class_='discussion-sidebar-item sidebar-labels js-discussion-sidebar-item').text.strip().split('\n'))

'Labels;   ; ;     None yet'

And that's how it is done:

In [0]:
# хоба!
'; '.join(soup_test.find(class_='discussion-sidebar-item sidebar-labels js-discussion-sidebar-item').text.strip('\n\n    Labels\n  \n\n').split('\n'))

'None yet'

#### closed at
Для closed at:
- сначала разбить по \n, получается список с одним элементом
- взять этот список и очистить от \n и лишних пробелов в начале/конце


This function is used to clean and extract data about who and when closed the issue. 
- split into 3 elements is NOT used in final version as some issues are still open --> now output is splitted by ;

In [0]:
who_did, what_did, when_did = clean_closed_at(soup_test_tf)
who_did

'actuallyykatie'

In [0]:
# tests
print('Multiple reviewers ---> '+ str(clean_closed_at(soup_test_tf)))
print('Issue with one assignee ---> ' + str(clean_closed_at(soup_test)))
print('Single reviewer ---> ' + str(clean_closed_at(soup_tf)))

Multiple reviewers ---> ('actuallyykatie', 'closed this', 'Nov 22, 2018')
Issue with one assignee ---> ('actuallyykatie', 'closed this', 'Nov 22, 2018')
Single reviewer ---> ('actuallyykatie', 'closed this', 'Nov 22, 2018')


In [0]:
datetime.strptime('Nov 22, 2018', '%b %d, %Y').strftime('%d/%m/%Y')

'22/11/2018'

**Updated example **of `clean_closed_at()`:

In [0]:
clean_closed_at(soup_test)

'actuallyykatie; closed this; Nov 22 2018'

#### comments processing
Processing comments is the most fun (no) part. Here we need to remove \n, commas and some other irrelevant stuff

In [0]:
#'comment_author' : [x.split('    ')[3] for x in clean_comments(soup_test)],
#'comment_date' : [x.split('    ')[6] for x in clean_comments(soup_test)],
#'comment_text' : [x.split('    ')[8] for x in clean_comments(soup_test)],  

['Collaborator', 'Collaborator', 'Collaborator', 'Collaborator']

It's clear that comments section contains information about the status of the user - it may be collaborator, author of the issue, contributor, member or maybe smth else.

In [0]:
clean_comments(soup_test)  # old

['Collaborator    actuallyykatie   commented    Nov 18 2018     Сделать стартовую страницу',
 'Collaborator   Author  actuallyykatie   commented    Nov 22 2018     Осталось:  доделать блок с жанрами добавить пару кнопок подправить обложки  Непонятно как:  пофиксить левое меню чтобы оно не двигалось',
 'Collaborator   Author  actuallyykatie   commented    Nov 22 2018     Сделано:  обновлен блок с жанрами обновлены данные (авторы названия обложки) плейлистов и треков теперь всё на своих местах  Надо бы потом доработать:  обрезка изображений раздел с жанрами тень кнопки',
 'Collaborator   Author  actuallyykatie   commented    Nov 22 2018    уменьшить размер верхнего бара и поиска']

In [0]:
clean_comments(soup_test_tf) # old

['ageron   commented    Sep 12 2016     This PR makes it possible to use the download_and_convert_data.py script with python 3.  It also adds support for CIFAR100. Regarding python 3 the problems were essentially:  import cPickle => from six.moves import cPickle cPickle.load(f) => cPickle.load(f encoding="bytes") use strings instead of byte strings in several places  Regarding CIFAR100 it\'s basically the same format as CIFAR10 so why not add it?',
 "googlebot   commented    Sep 12 2016     Thanks for your pull request.  It looks like this may be your first contribution to a Google open source project. Before we can look at your pull request you'll need to sign a Contributor License Agreement (CLA). 📝 Please visit https://cla.developers.google.com/ to sign. Once you've signed please reply here (e.g. I signed it!) and we'll verify.  Thanks. If you've already signed a CLA it's possible we don't have your GitHub username or you're using a different email address.  Check your existing CLA 

In [0]:
get_comments_df(soup_test)

,issue_title,comment_number,comments,reactions
0,1st page design,1,Collaborator actuallyykatie commented ...,[ ]
1,1st page design,2,Collaborator Author actuallyykatie commen...,[ ]
2,1st page design,3,Collaborator Author actuallyykatie commen...,[ ]
3,1st page design,4,Collaborator Author actuallyykatie commen...,[ ]


----

### examples

In [0]:
parse_issue('https://github.com/Kislyonkova/PSA-group-project/issues/1')

["['Kislyonkova', 'PSA-group-project', '1st page design', '1', 'Closed', 'actuallyykatie', 'None yet', '18/11/2018', 'actuallyykatie', 'closed this', '22/11/2018', '1 participant', 1, 'Collaborator    actuallyykatie   commented    Nov 18 2018     Сделать стартовую страницу', ['\\n']]",
 "['Kislyonkova', 'PSA-group-project', '1st page design', '1', 'Closed', 'actuallyykatie', 'None yet', '18/11/2018', 'actuallyykatie', 'closed this', '22/11/2018', '1 participant', 2, 'Collaborator   Author  actuallyykatie   commented    Nov 22 2018     Осталось:  доделать блок с жанрами добавить пару кнопок подправить обложки  Непонятно как:  пофиксить левое меню чтобы оно не двигалось', ['\\n']]",
 "['Kislyonkova', 'PSA-group-project', '1st page design', '1', 'Closed', 'actuallyykatie', 'None yet', '18/11/2018', 'actuallyykatie', 'closed this', '22/11/2018', '1 participant', 3, 'Collaborator   Author  actuallyykatie   commented    Nov 22 2018     Сделано:  обновлен блок с жанрами обновлены данные (авто

In [0]:
# Pool tells how many at a time
with Pool(10) as p:
  # pass parser function and urls to parse
  records = p.map(parse_issue, ['https://github.com/Kislyonkova/PSA-group-project/issues/1',
 'https://github.com/Kislyonkova/PSA-group-project/issues/2'])

Still needs a bit of cleaning but it's better than nothing c:

In [0]:
pd.DataFrame(flatten(records), columns=['everything']).everything.str.split(expand=True, pat=', ', n = 14)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,['Kislyonkova','PSA-group-project','1st page design','1','Closed','actuallyykatie','None yet','18/11/2018','actuallyykatie','closed this','22/11/2018','1 participant',"1, 'Collaborator actuallyykatie commented..."
1,['Kislyonkova','PSA-group-project','1st page design','1','Closed','actuallyykatie','None yet','18/11/2018','actuallyykatie','closed this','22/11/2018','1 participant',"2, 'Collaborator Author actuallyykatie co..."
2,['Kislyonkova','PSA-group-project','1st page design','1','Closed','actuallyykatie','None yet','18/11/2018','actuallyykatie','closed this','22/11/2018','1 participant',"3, 'Collaborator Author actuallyykatie co..."
3,['Kislyonkova','PSA-group-project','1st page design','1','Closed','actuallyykatie','None yet','18/11/2018','actuallyykatie','closed this','22/11/2018','1 participant',"4, 'Collaborator Author actuallyykatie co..."
4,['Kislyonkova','PSA-group-project','wiki page','2','Closed','Kislyonkova','None yet','18/11/2018','actuallyykatie','closed this','22/11/2018','2 participants',"1, 'Collaborator actuallyykatie commented..."


In [0]:
with Pool(10) as p:
  # pass parser function and urls to parse
  records = p.map(parse_issue, get_issue_links('Kislyonkova/PSA-group-project'))

In [0]:
pd.DataFrame(flatten(records), columns=['everything']).everything.str.split(expand=True, pat=', ', n = 14)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,['Kislyonkova','PSA-group-project','1st page design','1','Closed','actuallyykatie','None yet','18/11/2018','actuallyykatie','closed this','22/11/2018','1 participant',1,'Collaborator actuallyykatie commented ...,['\n']]
1,['Kislyonkova','PSA-group-project','1st page design','1','Closed','actuallyykatie','None yet','18/11/2018','actuallyykatie','closed this','22/11/2018','1 participant',2,'Collaborator Author actuallyykatie comme...,['\n']]
2,['Kislyonkova','PSA-group-project','1st page design','1','Closed','actuallyykatie','None yet','18/11/2018','actuallyykatie','closed this','22/11/2018','1 participant',3,'Collaborator Author actuallyykatie comme...,['\n']]
3,['Kislyonkova','PSA-group-project','1st page design','1','Closed','actuallyykatie','None yet','18/11/2018','actuallyykatie','closed this','22/11/2018','1 participant',4,'Collaborator Author actuallyykatie comme...,['\n']]
4,['Kislyonkova','PSA-group-project','wiki page','2','Closed','Kislyonkova','None yet','18/11/2018','actuallyykatie','closed this','22/11/2018','2 participants',1,'Collaborator actuallyykatie commented ...,['\n']]
5,['Kislyonkova','PSA-group-project','database','3','Closed','aemulus-b','None yet','18/11/2018','actuallyykatie','closed this','22/11/2018','2 participants',1,'Collaborator actuallyykatie commented ...,['\n']]
6,['Kislyonkova','PSA-group-project',"""Description of application's items""",'4','Closed','Kislyonkova','None yet','28/11/2018','actuallyykatie','closed this','22/11/2018','1 participant',1,"""Owner Kislyonkova commented Nov 28 20...",['\n']]
7,['Kislyonkova','PSA-group-project','Fix users database','5','Closed','aemulus-b','None yet','11/12/2018','actuallyykatie','closed this','22/11/2018','3 participants',1,'Collaborator actuallyykatie commented ...,['\n']]
8,['Kislyonkova','PSA-group-project','Fix users database','5','Closed','aemulus-b','None yet','11/12/2018','actuallyykatie','closed this','22/11/2018','3 participants',2,'Owner Kislyonkova commented Dec 14 20...,'\n\n\n\n🎉\n 1\n \n\n']
9,['Kislyonkova','PSA-group-project','Fix users database','5','Closed','aemulus-b','None yet','11/12/2018','actuallyykatie','closed this','22/11/2018','3 participants',3,'Collaborator Author actuallyykatie comme...,'\n\n\n\n🎉\n 1\n \n\n']


# Collect!
And here the data is collected!

### Top Fastest growing open source projects
1.	[MicrosoftDocs/azure-docs](https://github.com/MicrosoftDocs/azure-docs)	-- 1,885 open 14,990 closed
2.	[pytorch/pytorch](https://github.com/pytorch/pytorch) -- 2,280 open + 5,472 closed
3.	[godotengine/godot](https://github.com/godotengine/godot) 4,578 open + 12,231 closed
4.	[nuxt/nuxt.js](https://github.com/nuxt/nuxt.js)	129 open + 3,950 closed
5.	[ethereum/go-ethereum](https://github.com/ethereum/go-ethereum)	-- 578 open + 3,608 closed
6.	[wix/react-native-navigation](https://github.com/wix/react-native-navigation)	-- 153 open + 3,338 closed
7.	[spyder-ide/spyder](https://github.com/spyder-ide/spyder)	-- 700 open + 6,881 closed
8.	[tensorflow/models](https://github.com/tensorflow/models)	-- 1,183 open + 2,889 closed  // 359 open + 1,997 closed PRs
9.	[home-assistant/home-assistant	](https://github.com/home-assistant/home-assistant) -- 922 open + 8,331 closed
10.	[MarlinFirmware/Marlin](https://github.com/MarlinFirmware/Marlin) -- 719 open + 6,643 closed


#### tensorflow/models
Time: 20 min (without pool & map - ~3 hours!)

In [0]:
%%time
with Pool(10) as p:
  # pass parser function and urls to parse
  records_tf = p.map(parse_issue, get_issue_links('tensorflow/models'))

CPU times: user 1.09 s, sys: 707 ms, total: 1.8 s
Wall time: 20min 58s


In [0]:
len(records_tf)

6669

In [0]:
pd.DataFrame(flatten(records_tf), columns=['everything']).everything.str.split(expand=True, pat=', ', n = 13).to_csv(PATH_TO_DATA + 'tf.csv')

In [0]:
flatten(records_tf)

In [0]:
d_test = pd.read_csv(PATH_TO_DATA + 'tf.csv', index_col = 0)
d_test.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,['tensorflow','models','initial commit simple separated models','1','Merged','No; reviews','None yet','19/02/2016','','4 participants',1,'Contributor snurkabill commented Feb ...,'\n']
1,['tensorflow','models','initial commit simple separated models','1','Merged','No; reviews','None yet','19/02/2016','','4 participants',2,"""Member martinwicke commented Mar 8 20...",'\n']
2,['tensorflow','models','initial commit simple separated models','1','Merged','No; reviews','None yet','19/02/2016','','4 participants',3,"""Member martinwicke commented Mar 8 20...",'\n']


#### MicrosoftDocs/azure-docs 
Time: 1 hour 36 min

In [0]:
%%time
with Pool(10) as p:
  # pass parser function and urls to parse
  records_ms_ad = p.map(parse_issue, get_issue_links('MicrosoftDocs/azure-docs'))

CPU times: user 4.94 s, sys: 2.49 s, total: 7.43 s
Wall time: 1h 36min 56s


In [0]:
len(flatten(records_ms_ad))

146531

In [0]:
pd.DataFrame(flatten(records_ms_ad), columns=['everything']).everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'MicrosoftDocs_azure-docs.csv')

#### pytorch/pytorch
Time: 1 hour 17 min

In [0]:
%%time
with Pool(10) as p:
  records_pytorch = p.map(parse_issue, get_issue_links('pytorch/pytorch'))

CPU times: user 4.01 s, sys: 2.1 s, total: 6.11 s
Wall time: 1h 17min 22s


In [0]:
pd.DataFrame(flatten(records_pytorch), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'pytorch_pytorch.csv')

In [0]:
ptrch = pd.read_csv(PATH_TO_DATA + 'pytorch_pytorch.csv', index_col = 0)
ptrch.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,['pytorch','pytorch','Matrix multiplication operator','1','Closed','No; one; assigned','None yet','15/08/2016','soumith; closed this; Aug 24 2016','4 participants',1,"""Contributor bartvm commented Aug 15 2...",'\n\n\n\n👍\n 11\n \n\n']
1,['pytorch','pytorch','Matrix multiplication operator','1','Closed','No; one; assigned','None yet','15/08/2016','soumith; closed this; Aug 24 2016','4 participants',2,'alexbw commented Aug 16 2016 A good ...,'\n\n\n\n😄\n 1\n \n\n']
2,['pytorch','pytorch','Matrix multiplication operator','1','Closed','No; one; assigned','None yet','15/08/2016','soumith; closed this; Aug 24 2016','4 participants',3,"""Member apaszke commented Aug 16 2016 ...",'\n\n\n\n👍\n 2\n \n\n']


#### godotengine/godot 
Time: 1 hour 29 min

In [0]:
%%time
with Pool(10) as p:
  records_gg = p.map(parse_issue, get_issue_links('godotengine/godot'))

CPU times: user 4.81 s, sys: 2.37 s, total: 7.18 s
Wall time: 1h 29min 9s


In [0]:
pd.DataFrame(flatten(records_gg), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'godotengine_godot.csv')

#### nuxt/nuxt.js 
Time: 17 min

In [0]:
# error fixed
%%time
with Pool(10) as p:
  records_nuxt = p.map(parse_issue, get_issue_links('nuxt/nuxt.js'))

CPU times: user 1.01 s, sys: 891 ms, total: 1.9 s
Wall time: 17min 43s


In [0]:
pd.DataFrame(flatten(records_nuxt), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'nuxt_nuxt_js.csv')

In [0]:
nuxtjs = pd.read_csv(PATH_TO_DATA + 'nuxt_nuxt_js.csv', index_col = 0)
nuxtjs.head(3)
# featur? --> feature ???????

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,['nuxt','nuxt.js','Add pages/_app.vue','1','Closed','Atinux','type: featur','09/11/2016','Atinux; closed this; Nov 21 2016','1 participant',1,'Member Atinux commented Nov 9 2016 •...,['\n']]
1,['nuxt','nuxt.js','Add pages/_app.vue','1','Closed','Atinux','type: featur','09/11/2016','Atinux; closed this; Nov 21 2016','1 participant',2,"""Member Author Atinux commented Nov 21...",['\n']]
2,['nuxt','nuxt.js','Add pages/_app.vue','1','Closed','Atinux','type: featur','09/11/2016','Atinux; closed this; Nov 21 2016','1 participant',3,'lock bot commented Nov 5 2018 This t...,['\n']]


#### ethereum/go-ethereum 
Time: ?

In [0]:
%%time
with Pool(10) as p:
  records_eth = p.map(parse_issue, get_issue_links('ethereum/go-ethereum'))

In [0]:
pd.DataFrame(flatten(records_eth), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'ethereum_go_ethereum.csv')

#### wix/react-native-navigation 
Time: 15 minutes

In [0]:
%%time
with Pool(10) as p:
  records_react = p.map(parse_issue, get_issue_links('wix/react-native-navigation'))

CPU times: user 899 ms, sys: 929 ms, total: 1.83 s
Wall time: 15min 17s


In [0]:
pd.DataFrame(flatten(records_react), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'wix_react_native_navigation.csv')

#### spyder-ide/spyder 
Time:  ?


In [0]:
%%time
with Pool(4) as p:
  records_spyder = p.map(parse_issue, get_issue_links('spyder-ide/spyder'))

In [0]:
pd.DataFrame(flatten(records_spyder), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'spyder_ide_spyder.csv')

#### home-assistant/home-assistant 
Time: 1 hour 24 min

In [0]:
%%time
with Pool(10) as p:
  records_ha = p.map(parse_issue, get_issue_links('home-assistant/home-assistant'))

CPU times: user 5.87 s, sys: 2.52 s, total: 8.39 s
Wall time: 1h 24min 24s


In [0]:
len(records_ha)

23464

In [0]:
pd.DataFrame(flatten(records_ha), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'home_assistant.csv')

#### MarlinFirmware/Marlin 
Time: 1 hour 32 min

In [0]:
%%time
with Pool(4) as p:  # 4 - bad idea? :)))))))
  records_marlin = p.map(parse_issue, get_issue_links('MarlinFirmware/Marlin'))

CPU times: user 6.05 s, sys: 2.14 s, total: 8.19 s
Wall time: 1h 32min 55s


In [0]:
len(records_marlin)

13833

In [0]:
pd.DataFrame(flatten(records_marlin), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'marlin.csv')

***

### Top open source projects
1.	[Microsoft/vscode](https://github.com/Microsoft/vscode)	-- 5,226 open + 61,287 closed
2.	[facebook/react-native](https://github.com/facebook/react-native) -- 414 open + 15,542 closed
3.	[tensorflow/tensorflow](https://github.com/tensorflow/tensorflow)	-- 2,035 open + 14,952 closed
4.	[angular/angular-cli](https://github.com/angular/angular-cli)	-- 666 open + 9400 closed
5.	[MicrosoftDocs/azure-docs	](https://github.com/MicrosoftDocs/azure-docs) -- 1885 open + 14988 closed
6.	[angular/angular](https://github.com/angular/angular)	-- 2,255 open + 15,177 closed
7.	[ansible/ansible](https://github.com/ansible/ansible)	-- 3,877 opened + 18,453 closed
8.	[kubernetes/kubernetes](https://github.com/kubernetes/kubernetes) -- 2,166 open + 27,583 closed
9.	[npm/npm](https://github.com/npm/npm)	-- 2,166 open + 16,507 closed
10.	[DefinitelyTyped/DefinitelyTyped](https://github.com/DefinitelyTyped/DefinitelyTyped)	-- 2,651 open + 2,744 closed


#### Microsoft/vscode 
Time: ~5 hours


In [0]:
%%time
with Pool(10) as p:
  records_vscode = p.map(parse_issue, get_issue_links('Microsoft/vscode'))

In [0]:
pd.DataFrame(flatten(records_vscode), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'Microsoft_vscode.csv')

#### facebook/react-native 
Time: 1 hour 21 min

In [0]:
%%time
with Pool(10) as p:
  records_react = p.map(parse_issue, get_issue_links('facebook/react-native'))

CPU times: user 5.42 s, sys: 2.48 s, total: 7.9 s
Wall time: 1h 21min 49s


In [0]:
pd.DataFrame(flatten(records_react), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'facebook_react_native.csv')

In [0]:
len(records_react), len(flatten(records_react))

(24653, 172190)

In [0]:
pd.DataFrame(flatten(records_react), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,['facebook','react-native','nil','1','Closed','No; reviews','None yet','28/01/2015','sophiebits; closed this; Jan 29 2015','3 participants',1,'robz commented Jan 28 2015 No descrip...,['\n']]
1,['facebook','react-native','nil','1','Closed','No; reviews','None yet','28/01/2015','sophiebits; closed this; Jan 29 2015','3 participants',2,'facebook-github-bot commented Mar 31 201...,['\n']]
2,['facebook','react-native','`pthread.h` not found','2','Closed','No; one; assigned','Resolution: Locked','30/01/2015','vjeux; closed this; Jan 31 2015','4 participants',1,'jlongster commented Jan 30 2015 To b...,['\n']]
3,['facebook','react-native','`pthread.h` not found','2','Closed','No; one; assigned','Resolution: Locked','30/01/2015','vjeux; closed this; Jan 31 2015','4 participants',2,'Contributor vjeux commented Jan 30 20...,['\n']]
4,['facebook','react-native','`pthread.h` not found','2','Closed','No; one; assigned','Resolution: Locked','30/01/2015','vjeux; closed this; Jan 31 2015','4 participants',3,'Contributor sophiebits commented Jan ...,['\n']]


#### tensorflow/tensorflow 


In [0]:
%%time
with Pool(10) as p:
  records_tftf = p.map(parse_issue, get_issue_links('tensorflow/tensorflow'))

CPU times: user 10.2 s, sys: 4.35 s, total: 14.5 s
Wall time: 2h 43min 17s


In [0]:
pd.DataFrame(flatten(records_tftf), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'tensorflow_tensorflow.csv')

#### angular/angular-cli
Time: 43 min


In [0]:
%%time
with Pool(10) as p:
  records_angcli = p.map(parse_issue, get_issue_links('angular/angular-cli'))

CPU times: user 2.78 s, sys: 1.16 s, total: 3.95 s
Wall time: 43min 10s


In [0]:
len(records_angcli), len(flatten(records_angcli))

(14301, 75811)

In [0]:
pd.DataFrame(flatten(records_angcli), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'angular_angular_cli.csv')

#### MicrosoftDocs/azure-docs
- already collected, also is a fast growing repo (member of both tops)

In [0]:
#%%time
#with Pool(10) as p:
#  records_azdoc = p.map(parse_issue, get_issue_links('MicrosoftDocs/azure-docs'))

In [0]:
#pd.DataFrame(flatten(records_azdoc), columns=['everything'])\
#.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'MicrosoftDocs_azure_docs.csv')

#### angular/angular 
Time: ?

In [0]:
%%time
with Pool(10) as p:
  records_ang = p.map(parse_issue, get_issue_links('angular/angular'))

In [0]:
pd.DataFrame(flatten(records_ang), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'angular_angular.csv')

#### ansible/ansible
Time: 2 hours 52 min

In [0]:
%%time
with Pool(10) as p:
  records_ansib = p.map(parse_issue, get_issue_links('ansible/ansible'))

CPU times: user 9.69 s, sys: 4.28 s, total: 14 s
Wall time: 2h 52min 30s


In [0]:
pd.DataFrame(flatten(records_ansib), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'ansible_ansible.csv')

#### kubernetes/kubernetes 
Time:  4 hours 26 min

In [0]:
%%time
with Pool(10) as p:
  records_kub = p.map(parse_issue, get_issue_links('kubernetes/kubernetes'))

CPU times: user 15.6 s, sys: 6.8 s, total: 22.4 s
Wall time: 4h 26min 19s


In [0]:
len(records_kub), len(flatten(records_kub))

(77182, 790596)

In [0]:
pd.DataFrame(flatten(records_kub), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'kubernetes_kubernetes.csv')

#### npm/npm
Time:  51 min

In [0]:
%%time
with Pool(10) as p:
  records_npm = p.map(parse_issue, get_issue_links('npm/npm'))

CPU times: user 3.56 s, sys: 2.04 s, total: 5.6 s
Wall time: 51min 54s


In [0]:
len(records_npm)

21209

In [0]:
pd.DataFrame(flatten(records_npm), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'npm_npm.csv')


#### DefinitelyTyped/DefinitelyTyped
Time: 2 hours 6 min

In [0]:
%%time
with Pool(10) as p:
  records_deftyp = p.map(parse_issue, get_issue_links('DefinitelyTyped/DefinitelyTyped'))

CPU times: user 9.99 s, sys: 3.99 s, total: 14 s
Wall time: 2h 6min 22s


In [0]:
len(records_deftyp), len(flatten(records_deftyp))

(35043, 145835)

In [0]:
pd.DataFrame(flatten(records_deftyp), columns=['everything'])\
.everything.str.split(expand=True, pat=', ', n = 14).to_csv(PATH_TO_DATA + 'DefinitelyTyped_DefinitelyTyped.csv')